# Notebook Description

This notebook shows how to simulate a two-dimensional shock-bubble interaction using MFC.
The simulation shows a helium bubble in air being impinged with a Mach 1.5 shockwave.
This is a canonical test problem in compressible fluid dynamics with ample experimental and computational results to compare to.
This notebook highlights how Cybershuttle can be used to dispatch software to multiple different systems from one notebook.

# Request Resources

In [1]:
%pip install -qU "airavata-python-sdk[notebook]"
import airavata_jupyter_magic

%authenticate

# Request resource on SDSC Expanse
# %request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=expanse:shared

# Request resource on Jetstream 2
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=NeuroData25VC1:cloud

%wait_for_runtime hpc_cpu --live
%copy_data source=local:case.py target=hpc_cpu:case.py 
%switch_runtime hpc_cpu

Note: you may need to restart the kernel to use updated packages.

Loaded airavata_jupyter_magic (2.1.4.post4) 
(current runtime = local)

  %authenticate                              -- Authenticate to access high-performance runtimes.
  %request_runtime <rt> [args]               -- Request a runtime named <rt> with configuration <args>.
                                                Call multiple times to request multiple runtimes.
  %restart_runtime <rt>                      -- Restart runtime <rt> if it hangs. This will clear all variables.
  %stop_runtime <rt>                         -- Stop runtime <rt> when no longer needed.
  %wait_for_runtime <rt>                     -- Wait for runtime <rt> to be ready.
  %switch_runtime <rt>                       -- Switch the active runtime to <rt>. All subsequent cells will run here.
  %%run_on <rt>                              -- Force a cell to always execute on <rt>, regardless of the active runtime.
  %stat_runtime <rt>               

Output()

Authenticated.

Requesting runtime=hpc_cpu...
[NeuroData25VC1:cloud, 60 Minutes, 1 Node(s), 4 CPU(s), 0 GPU(s), 4000 MB RAM, 0 MB VRAM]
* modules=[]
* libraries=['python=3.10', 'pip', 'make', 'cmake', 'compilers', 'openmpi', 'wget']
* pip=[]
* mounts=[]
Requested runtime=hpc_cpu
Request successful: runtime=hpc_cpu


Output()

local:/var/folders/_n/fcf6nx4j67gbbt4_8mjqxdc80000gn/T/connection_rv_l7mo3.json --> hpc_cpu:connection_rv_l7mo3.json... [200]
started proc_name=hpc_cpu_kernel on rt=hpc_cpu. pid=2812
forwarding ports=[27300, 27301, 27302, 27303, 27304]
hpc_cpu:27300 -> access via 18.118.140.230:10000
hpc_cpu:27301 -> access via 18.118.140.230:10001
hpc_cpu:27302 -> access via 18.118.140.230:10002
hpc_cpu:27303 -> access via 18.118.140.230:10003
hpc_cpu:27304 -> access via 18.118.140.230:10004
started ipykernel tunnels for hpc_cpu at 18.118.140.230
started ipykernel client for hpc_cpu
Remote Jupyter kernel launched and connected for runtime=hpc_cpu.
copying local:case.py to hpc_cpu:case.py
local:case.py --> hpc_cpu:case.py... [200]
Switched to runtime=hpc_cpu.


# Run Job on Expanse

%%bash

export MFC_Root="$HOME/apps/MFC/MFC-Wilfong"
. $MFC_Root/mfc.sh load -c e -m c
module load cmake/3.21.4
$MFC_Root/mfc.sh run case.py --case-optimization --no-gpu --mpi -b mpirun -N 1 -n 4 -j 4

# Run on Jetstream 2

In [2]:
%%bash

export MFC_Root="$HOME/apps/MFC-Wilfong"
module load gnu9/9.4.0 openmpi4/4.1.1
$MFC_Root/mfc.sh run case.py --case-optimization --no-gpu --mpi -b mpirun -N 1 -n 4 -j 4

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...


cat: CMakeLists.txt: No such file or directory


mfc: OK > (venv) Entered the Python 3.11.11 virtual environment (>= 3.9).

      .=++*:          -+*+=.        | exouser@nsworkshopcpuvc1-compute-0.novalocal [Linux]
     :+   -*-        ==   =* .      | ----------------------------------------------------
   :*+      ==      ++    .+-       | 
  :*##-.....:*+   .#%+++=--+=:::.   | --jobs 4
  -=-++-======#=--**+++==+*++=::-:. | --mpi --no-gpu --no-debug --no-gcov --no-unified --no-single
 .:++=----------====+*= ==..:%..... | --targets pre_process, simulation, and post_process
  .:-=++++===--==+=-+=   +.  :=     | 
  +#=::::::::=%=. -+:    =+   *:    | ----------------------------------------------------------
 .*=-=*=..    :=+*+:      -...--    | $ ./mfc.sh (build, run, test, clean, count, packer) --help

 Acquiring /home/exouser/cybershuttle/workspace/PROCESS_4b9c9c2e-1e0d-44b7-84cd-e67abbaf2be8/case.py...
 Acquiring /home/exouser/cybershuttle/workspace/PROCESS_4b9c9c2e-1e0d-44b7-84cd-e67abbaf2be8/case.py...
 Build | syscheck, syschec

/home/exouser/apps/MFC-Wilfong/src/common/m_mpi_common.fpp:216:26:

  216 |                 sizes_glb(3) = p_glb + 1; sizes_loc(3) = p + 1
      |                          1
/home/exouser/apps/MFC-Wilfong/src/common/m_mpi_common.fpp:216:52:

  216 |                 sizes_glb(3) = p_glb + 1; sizes_loc(3) = p + 1
      |                                                    1


[  4%] Building Fortran object CMakeFiles/simulation.dir/fypp/simulation/m_mpi_proxy.fpp.f90.o
[  6%] Linking Fortran executable simulation
[100%] Built target simulation

 $ cmake --install /home/exouser/apps/MFC-Wilfong/build/staging/014f4af222

-- Install configuration: "Release"
-- Installing: /home/exouser/apps/MFC-Wilfong/build/install/014f4af222/bin/simulation

 Generating case.fpp.
   Writing a (new) custom case.fpp file.
 $ cmake --build /home/exouser/apps/MFC-Wilfong/build/staging/dd3ec89703 --target post_process --parallel 4 --config Release

[100%] Built target post_process

 $ cmake --install /home/exouser/apps/MFC-Wilfong/build/staging/dd3ec89703

-- Install configuration: "Release"
-- Installing: /home/exouser/apps/MFC-Wilfong/build/install/dd3ec89703/bin/post_process
-- Set non-toolchain portion of runtime path of "/home/exouser/apps/MFC-Wilfong/build/install/dd3ec89703/bin/post_process" to ""

 Run
   Using queue system Interactive.
   Using baked-in template for defau

...

 Pre-processing a 300x100x0 case on 4 rank(s)
 Processing patch           1
 Processing patch           2
 Processing patch           3
 Elapsed Time   5.4125999999999980E-002

mfc: OK > :) Running simulation:



+ mpirun -np 4 /home/exouser/apps/MFC-Wilfong/build/install/014f4af222/bin/simulation


 Simulating a case-optimized 300x100x0 case on 4 rank(s) on CPUs.
 [  0%]  Time step        1 of 1925 @ t_step = 0
 [  1%]  Time step        2 of 1925 @ t_step = 1
 [  1%]  Time step        3 of 1925 @ t_step = 2
 [  1%]  Time step        4 of 1925 @ t_step = 3
 [  1%]  Time step        5 of 1925 @ t_step = 4
 [  1%]  Time step        6 of 1925 @ t_step = 5
 [  1%]  Time step        7 of 1925 @ t_step = 6
 [  1%]  Time step        8 of 1925 @ t_step = 7
 [  1%]  Time step        9 of 1925 @ t_step = 8
 [  1%]  Time step       10 of 1925 @ t_step = 9
 [  1%]  Time step       11 of 1925 @ t_step = 10
 [  1%]  Time step       12 of 1925 @ t_step = 11
 [  1%]  Time step       13 of 1925 @ t_step = 12
 [  1%]  Time step       14 of 1925 @ t_step = 13
 [  1%]  Time step       15 of 1925 @ t_step = 14
 [  1%]  Time step       16 of 1925 @ t_step = 15
 [  1%]  Time step       17 of 1925 @ t_step = 16
 [  1%]  Time step       18 of 1925 @ t_step = 17
 [  1%]  Time step       19 of 1925 @ t_step

+ mpirun -np 4 /home/exouser/apps/MFC-Wilfong/build/install/dd3ec89703/bin/post_process


 Post-processing a 300x100x0 case on 4 rank(s)
 [  0%]  Saving        1 of 102 @ t_step = 0
 [  1%]  Saving        2 of 102 @ t_step = 19
 [  2%]  Saving        3 of 102 @ t_step = 38
 [  3%]  Saving        4 of 102 @ t_step = 57
 [  4%]  Saving        5 of 102 @ t_step = 76
 [  5%]  Saving        6 of 102 @ t_step = 95
 [  6%]  Saving        7 of 102 @ t_step = 114
 [  7%]  Saving        8 of 102 @ t_step = 133
 [  8%]  Saving        9 of 102 @ t_step = 152
 [  9%]  Saving       10 of 102 @ t_step = 171
 [ 10%]  Saving       11 of 102 @ t_step = 190
 [ 11%]  Saving       12 of 102 @ t_step = 209
 [ 12%]  Saving       13 of 102 @ t_step = 228
 [ 13%]  Saving       14 of 102 @ t_step = 247
 [ 14%]  Saving       15 of 102 @ t_step = 266
 [ 15%]  Saving       16 of 102 @ t_step = 285
 [ 16%]  Saving       17 of 102 @ t_step = 304
 [ 17%]  Saving       18 of 102 @ t_step = 323
 [ 18%]  Saving       19 of 102 @ t_step = 342
 [ 19%]  Saving       20 of 102 @ t_step = 361
 [ 20%]  Saving     

# Compress Database Files For Transfer

In [3]:
!tar -czf silo.tar.gz silo_hdf5

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
cell finished on hpc_cpu.


# Transfer Files

In [4]:
%copy_data source=hpc_cpu:silo.tar.gz target=local:silo.tar.gz

copying hpc_cpu:silo.tar.gz to local:silo.tar.gz
GET http://api.gateway.cybershuttle.org:8050/list/live/PROCESS_4b9c9c2e-1e0d-44b7-84cd-e67abbaf2be8/silo.tar.gz
GET http://api.gateway.cybershuttle.org:8050/download/live/PROCESS_4b9c9c2e-1e0d-44b7-84cd-e67abbaf2be8/silo.tar.gz
local:/Users/yasith/projects/artisan/cybershuttle-catalog/12-mfc-test/silo.tar.gz <-- hpc_cpu:silo.tar.gz... 

In [5]:
%stop_runtime hpc_cpu

Terminated runtime=hpc_cpu. state={'experimentId': 'CS_Agent_6b1ce399-3715-4950-80dc-2c23cf82e17a', 'terminated': True}
